# Notebook to find ideal prophet parameters

In [10]:
import pandas as pd
import numpy as np
from prophet import Prophet
import glob
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [11]:
# Load all household data files into a dictionary
def load_household_data():
    # Get all CSV files from household_data directory
    csv_files = glob.glob("../../../../data/household_data/*.csv")
    
    household_dict = {}
    
    for file_path in csv_files:
        # Extract user ID from filename
        filename = os.path.basename(file_path)
        user_id = filename.replace('user_data_', '').replace('.csv', '')
        
        # Load CSV and store in dictionary
        df = pd.read_csv(file_path)
        
        # Resample to hourly frequency using sums
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df.set_index('datetime')
        
        # drop status column if it exists
        df.drop("status", axis=1, inplace=True, errors='ignore')
        
        # Resample numeric columns (value) to hourly frequency using sum
        df_hourly = df[['value']].resample('h').sum()
        
        
        # Reset index to have datetime as a column again
        df_hourly = df_hourly.reset_index()
        
        household_dict[user_id] = df_hourly
    
    print(f"Loaded {len(household_dict)} household datasets (resampled to hourly frequency)")
    return household_dict

# Load the household data
household_data = load_household_data()

Loaded 106 household datasets (resampled to hourly frequency)


# Calculate quality measures for different prophet parameters

In [12]:
# Define parameter ranges for testing
# changepoint_prior_scale: Controls flexibility of trend changes
# - Lower values (0.001): Conservative, less flexible trend (good for stable patterns)
# - Medium values (0.05): Moderate flexibility (Prophet's default)
# - Higher values (0.5): More aggressive, captures more trend changes

changepoint_prior_scales = [1.0, 0.25, 0.5] # Conservative, Moderate, Aggressive

# seasonality_prior_scale: Controls strength of seasonality component
# - Lower values (0.1): Conservative, weaker seasonality fit
# - Medium values (1.0): Moderate seasonality strength
# - Higher values (10.0): Stronger seasonality fit (good for pronounced patterns)

seasonality_prior_scales = [2.0, 1.0, 0.5]  # Conservative, Moderate, Strong



print(f"Testing {len(changepoint_prior_scales)} × {len(seasonality_prior_scales)} = {len(changepoint_prior_scales) * len(seasonality_prior_scales)} parameter combinations")
print(f"\nChangepoint Prior Scales: {changepoint_prior_scales}")
print(f"Seasonality Prior Scales: {seasonality_prior_scales}")


Testing 3 × 3 = 9 parameter combinations

Changepoint Prior Scales: [1.0, 0.25, 0.5]
Seasonality Prior Scales: [2.0, 1.0, 0.5]


In [13]:
def forecast_prophet(df, days=30, changepoint_prior_scale=0.05, seasonality_prior_scale=1.0, yearly_seasonality=False):
    """
    Forecast energy usage using Prophet with configurable parameters
    
    Args:
        df: DataFrame with 'datetime' and 'value' columns
        days: Number of days to forecast
        changepoint_prior_scale: Flexibility of trend changes (default: 0.05)
        seasonality_prior_scale: Strength of seasonality (default: 1.0)
        yearly_seasonality: Whether to include yearly seasonality (default: False)
    
    Returns:
        Prophet forecast DataFrame and model
    """
    # Explicitly create a copy to avoid SettingWithCopyWarning
    df = df.copy()
    df["datetime"] = pd.to_datetime(df["datetime"])

    prophet_df = df.copy()
    prophet_df.rename(columns={'datetime': 'ds', 'value': 'y'}, inplace=True)
    
    # Drop status column if it exists (some datasets may not have it)
    prophet_df.drop(columns=['status'], inplace=True, errors='ignore')
    
    prophet_model = Prophet(
        daily_seasonality=True,
        yearly_seasonality=yearly_seasonality,
        weekly_seasonality=True,
        changepoint_prior_scale=changepoint_prior_scale,
        seasonality_prior_scale=seasonality_prior_scale,
        interval_width=0.95,
        growth="linear",
        seasonality_mode='additive'
    )

    prophet_model.fit(prophet_df)

    future = prophet_model.make_future_dataframe(periods=24*days, freq='h')
    forecast = prophet_model.predict(future)

    return forecast, prophet_model

In [16]:
def evaluate_forecast(actual_df, forecast_df, forecast_days=30):
    """
    Evaluate forecast accuracy by comparing predictions with actual values
    
    Args:
        actual_df: DataFrame with actual values (columns: datetime, value)
        forecast_df: Prophet forecast DataFrame
        forecast_days: Number of days that were forecasted
    
    Returns:
        Dictionary with MAE, RMSE, MAPE, total actual, total forecast, absolute and percentage differences
    """
    
    # Get the last N days of actual data for comparison
    actual_df = actual_df.copy()
    actual_df["datetime"] = pd.to_datetime(actual_df["datetime"])
    
    # Merge actual and forecast data
    forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])
    
    # Get only the forecasted period from the forecast
    split_date = actual_df['datetime'].iloc[-forecast_days * 24]
    forecast_period = forecast_df[forecast_df['ds'] >= split_date]
    
    # Merge on datetime
    merged = pd.merge(
        actual_df.rename(columns={'datetime': 'ds'}),
        forecast_period[['ds', 'yhat']],
        on='ds',
        how='inner'
    )
    
    if len(merged) == 0:
        return {'mae': float('inf'), 'rmse': float('inf'), 'mape': float('inf'), 
                'total_actual': 0, 'total_forecast': 0, 'absolute_diff': 0, 'percentage_diff': 0}
    
    # Calculate total energy consumption over the forecasted period
    total_actual = merged['value'].sum()
    total_forecast = merged['yhat'].sum()
    absolute_diff = abs(total_actual - total_forecast)
    percentage_diff = (absolute_diff / total_actual * 100) if total_actual > 0 else 0
    
    # Calculate metrics
    mae = mean_absolute_error(merged['value'], merged['yhat'])
    mse = mean_squared_error(merged['value'], merged['yhat'])
    rmse = np.sqrt(mse)  # Calculate RMSE manually for compatibility with older scikit-learn
    mape = (abs((merged['value'] - merged['yhat']) / merged['value']).mean()) * 100
    
    return {
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'n_points': len(merged),
        'total_actual_kwh': total_actual,
        'total_forecast_kwh': total_forecast,
        'absolute_diff_kwh': absolute_diff,
        'percentage_diff': percentage_diff
    }

In [21]:
def test_prophet_parameters(household_data, test_days=30, sample_size=10):
    """
    Test different Prophet parameter combinations across multiple households
    
    Args:
        household_data: Dictionary of household DataFrames
        test_days: Number of days to use for testing/validation
        sample_size: Number of households to test (None for all)
    
    Returns:
        DataFrame with results for each parameter combination
    """
    results = []
    
    # Sample households if specified
    household_ids = list(household_data.keys())
    if sample_size and sample_size < len(household_ids):
        import random
        household_ids = random.sample(household_ids, sample_size)
    
    total_combinations = len(changepoint_prior_scales) * len(seasonality_prior_scales)
    current_combo = 0
    
    print(f"Testing {total_combinations} parameter combinations on {len(household_ids)} households...")
    print("=" * 80)
    
    for cp_scale in changepoint_prior_scales:
        for s_scale in seasonality_prior_scales:
            current_combo += 1
            print(f"\n[{current_combo}/{total_combinations}] Testing: changepoint={cp_scale}, seasonality={s_scale}")
            
            combo_metrics = []
            
            for household_id in household_ids:
                    try:
                        df = household_data[household_id]
                        
                        # Skip if not enough data
                        if len(df) < test_days * 24 * 2:
                            continue
                        
                        # Split data: use last test_days for validation
                        split_idx = len(df) - (test_days * 24)
                        train_df = df.iloc[:split_idx]
                        
                        # Generate forecast
                        forecast, model = forecast_prophet(
                            train_df,
                            days=test_days,
                            changepoint_prior_scale=cp_scale,
                            seasonality_prior_scale=s_scale
                        )
                        
                        # Evaluate against actual data
                        metrics = evaluate_forecast(df, forecast, forecast_days=test_days)
                        combo_metrics.append(metrics)
                        
                    except Exception as e:
                        print(f"  Warning: Error processing household {household_id}: {str(e)}")
                        continue
                
                # Calculate average metrics for this combination
            if combo_metrics:
                avg_mae = sum(m['mae'] for m in combo_metrics) / len(combo_metrics)
                avg_rmse = sum(m['rmse'] for m in combo_metrics) / len(combo_metrics)
                avg_mape = sum(m['mape'] for m in combo_metrics) / len(combo_metrics)
                avg_total_actual = sum(m['total_actual_kwh'] for m in combo_metrics) / len(combo_metrics)
                avg_total_forecast = sum(m['total_forecast_kwh'] for m in combo_metrics) / len(combo_metrics)
                
                # Recalculate difference from averaged totals (not average of differences)
                avg_absolute_diff = abs(avg_total_actual - avg_total_forecast)
                avg_percentage_diff = (avg_absolute_diff / avg_total_actual * 100) if avg_total_actual > 0 else 0
                
                results.append({
                    'changepoint_prior_scale': cp_scale,
                    'seasonality_prior_scale': s_scale,
                    'avg_mae': avg_mae,
                    'avg_rmse': avg_rmse,
                    'avg_mape': avg_mape,
                    'avg_total_actual_kwh': avg_total_actual,
                    'avg_total_forecast_kwh': avg_total_forecast,
                    'avg_absolute_diff_kwh': avg_absolute_diff,
                    'avg_percentage_diff': avg_percentage_diff,
                    'n_households': len(combo_metrics)
                })
                
                print(f"  Results: MAE={avg_mae:.4f}, RMSE={avg_rmse:.4f}, MAPE={avg_mape:.2f}%")
                print(f"           Total: Actual={avg_total_actual:.2f} kWh, Forecast={avg_total_forecast:.2f} kWh, Diff={avg_absolute_diff:.2f} kWh ({avg_percentage_diff:.2f}%)")

    
    results_df = pd.DataFrame(results)
    return results_df.sort_values('avg_mae')

# Run Parameter Tests

In [24]:
# Run the parameter testing
# Adjust sample_size to test on more households (set to None for all)
results_df = test_prophet_parameters(household_data, test_days=30, sample_size=10)

# Display results sorted by best MAE
print("\n" + "=" * 80)
print("RESULTS - Sorted by Mean Absolute Error (MAE)")
print("=" * 80)
print(results_df.to_string(index=False))

13:42:00 - cmdstanpy - INFO - Chain [1] start processing


Testing 9 parameter combinations on 10 households...

[1/9] Testing: changepoint=1.0, seasonality=2.0


13:42:01 - cmdstanpy - INFO - Chain [1] done processing
13:42:02 - cmdstanpy - INFO - Chain [1] start processing
13:42:02 - cmdstanpy - INFO - Chain [1] done processing
13:42:03 - cmdstanpy - INFO - Chain [1] start processing
13:42:04 - cmdstanpy - INFO - Chain [1] done processing
13:42:05 - cmdstanpy - INFO - Chain [1] start processing
13:42:06 - cmdstanpy - INFO - Chain [1] done processing
13:42:07 - cmdstanpy - INFO - Chain [1] start processing
13:42:08 - cmdstanpy - INFO - Chain [1] done processing
13:42:09 - cmdstanpy - INFO - Chain [1] start processing
13:42:10 - cmdstanpy - INFO - Chain [1] done processing
13:42:11 - cmdstanpy - INFO - Chain [1] start processing
13:42:12 - cmdstanpy - INFO - Chain [1] done processing
13:42:13 - cmdstanpy - INFO - Chain [1] start processing
13:42:14 - cmdstanpy - INFO - Chain [1] done processing
13:42:15 - cmdstanpy - INFO - Chain [1] start processing
13:42:16 - cmdstanpy - INFO - Chain [1] done processing
13:42:17 - cmdstanpy - INFO - Chain [1] 

  Results: MAE=0.8211, RMSE=1.1569, MAPE=inf%
           Total: Actual=1153.70 kWh, Forecast=1064.21 kWh, Diff=89.48 kWh (7.76%)

[2/9] Testing: changepoint=1.0, seasonality=1.0


13:42:20 - cmdstanpy - INFO - Chain [1] done processing
13:42:21 - cmdstanpy - INFO - Chain [1] start processing
13:42:21 - cmdstanpy - INFO - Chain [1] done processing
13:42:22 - cmdstanpy - INFO - Chain [1] start processing
13:42:23 - cmdstanpy - INFO - Chain [1] done processing
13:42:24 - cmdstanpy - INFO - Chain [1] start processing
13:42:25 - cmdstanpy - INFO - Chain [1] done processing
13:42:26 - cmdstanpy - INFO - Chain [1] start processing
13:42:27 - cmdstanpy - INFO - Chain [1] done processing
13:42:28 - cmdstanpy - INFO - Chain [1] start processing
13:42:29 - cmdstanpy - INFO - Chain [1] done processing
13:42:30 - cmdstanpy - INFO - Chain [1] start processing
13:42:32 - cmdstanpy - INFO - Chain [1] done processing
13:42:32 - cmdstanpy - INFO - Chain [1] start processing
13:42:33 - cmdstanpy - INFO - Chain [1] done processing
13:42:34 - cmdstanpy - INFO - Chain [1] start processing
13:42:35 - cmdstanpy - INFO - Chain [1] done processing
13:42:36 - cmdstanpy - INFO - Chain [1] 

...

  Results: MAE=0.8193, RMSE=1.1561, MAPE=inf%
           Total: Actual=1153.70 kWh, Forecast=1058.93 kWh, Diff=94.76 kWh (8.21%)

[8/9] Testing: changepoint=0.5, seasonality=1.0


13:44:04 - cmdstanpy - INFO - Chain [1] done processing
13:44:05 - cmdstanpy - INFO - Chain [1] start processing
13:44:06 - cmdstanpy - INFO - Chain [1] done processing
13:44:07 - cmdstanpy - INFO - Chain [1] start processing
13:44:07 - cmdstanpy - INFO - Chain [1] done processing
13:44:08 - cmdstanpy - INFO - Chain [1] start processing
13:44:09 - cmdstanpy - INFO - Chain [1] done processing
13:44:10 - cmdstanpy - INFO - Chain [1] start processing
13:44:12 - cmdstanpy - INFO - Chain [1] done processing
13:44:13 - cmdstanpy - INFO - Chain [1] start processing
13:44:14 - cmdstanpy - INFO - Chain [1] done processing
13:44:15 - cmdstanpy - INFO - Chain [1] start processing
13:44:16 - cmdstanpy - INFO - Chain [1] done processing
13:44:17 - cmdstanpy - INFO - Chain [1] start processing
13:44:17 - cmdstanpy - INFO - Chain [1] done processing
13:44:18 - cmdstanpy - INFO - Chain [1] start processing
13:44:19 - cmdstanpy - INFO - Chain [1] done processing
13:44:20 - cmdstanpy - INFO - Chain [1] 

  Results: MAE=0.8201, RMSE=1.1569, MAPE=inf%
           Total: Actual=1153.70 kWh, Forecast=1058.12 kWh, Diff=95.58 kWh (8.28%)

[9/9] Testing: changepoint=0.5, seasonality=0.5


13:44:22 - cmdstanpy - INFO - Chain [1] done processing
13:44:23 - cmdstanpy - INFO - Chain [1] start processing
13:44:23 - cmdstanpy - INFO - Chain [1] done processing
13:44:24 - cmdstanpy - INFO - Chain [1] start processing
13:44:25 - cmdstanpy - INFO - Chain [1] done processing
13:44:26 - cmdstanpy - INFO - Chain [1] start processing
13:44:27 - cmdstanpy - INFO - Chain [1] done processing
13:44:28 - cmdstanpy - INFO - Chain [1] start processing
13:44:30 - cmdstanpy - INFO - Chain [1] done processing
13:44:30 - cmdstanpy - INFO - Chain [1] start processing
13:44:31 - cmdstanpy - INFO - Chain [1] done processing
13:44:32 - cmdstanpy - INFO - Chain [1] start processing
13:44:33 - cmdstanpy - INFO - Chain [1] done processing
13:44:34 - cmdstanpy - INFO - Chain [1] start processing
13:44:35 - cmdstanpy - INFO - Chain [1] done processing
13:44:36 - cmdstanpy - INFO - Chain [1] start processing
13:44:37 - cmdstanpy - INFO - Chain [1] done processing
13:44:38 - cmdstanpy - INFO - Chain [1] 

  Results: MAE=0.8193, RMSE=1.1561, MAPE=inf%
           Total: Actual=1153.70 kWh, Forecast=1058.43 kWh, Diff=95.27 kWh (8.26%)

RESULTS - Sorted by Mean Absolute Error (MAE)
 changepoint_prior_scale  seasonality_prior_scale  avg_mae  avg_rmse  avg_mape  avg_total_actual_kwh  avg_total_forecast_kwh  avg_absolute_diff_kwh  avg_percentage_diff  n_households
                    0.25                      2.0 0.817820  1.155704       inf             1153.6975             1049.700589             103.996911             9.014227            10
                    0.25                      0.5 0.817973  1.155142       inf             1153.6975             1053.120608             100.576892             8.717787            10
                    0.25                      1.0 0.818727  1.155530       inf             1153.6975             1054.465297              99.232203             8.601232            10
                    0.50                      0.5 0.819326  1.156126       inf             1

In [23]:
# Analyze the scale of your energy consumption data
print("=" * 80)
print("ENERGY CONSUMPTION DATA ANALYSIS")
print("=" * 80)

all_values = []
for household_id, df in household_data.items():
    all_values.extend(df['value'].tolist())

all_values = np.array(all_values)
all_values = all_values[all_values > 0]  # Remove zeros for better statistics

print(f"\nOverall Statistics:")
print(f"  Mean hourly consumption: {np.mean(all_values):.4f} kWh")
print(f"  Median hourly consumption: {np.median(all_values):.4f} kWh")
print(f"  Std Dev: {np.std(all_values):.4f} kWh")
print(f"  Min: {np.min(all_values):.4f} kWh")
print(f"  Max: {np.max(all_values):.4f} kWh")
print(f"  25th percentile: {np.percentile(all_values, 25):.4f} kWh")
print(f"  75th percentile: {np.percentile(all_values, 75):.4f} kWh")

# Calculate relative performance
best = results_df.iloc[0]
print(f"\n{'=' * 80}")
print("RELATIVE FORECAST PERFORMANCE")
print("=" * 80)
print(f"MAE as % of mean consumption: {(best['avg_mae'] / np.mean(all_values) * 100):.2f}%")
print(f"RMSE as % of std dev: {(best['avg_rmse'] / np.std(all_values) * 100):.2f}%")
print(f"MAPE: {best['avg_mape']:.2f}%")

ENERGY CONSUMPTION DATA ANALYSIS

Overall Statistics:
  Mean hourly consumption: 1.2118 kWh
  Median hourly consumption: 0.7080 kWh
  Std Dev: 1.5819 kWh
  Min: 0.0010 kWh
  Max: 34.7460 kWh
  25th percentile: 0.2980 kWh
  75th percentile: 1.5330 kWh

RELATIVE FORECAST PERFORMANCE
MAE as % of mean consumption: 26.25%
RMSE as % of std dev: 31.39%
MAPE: 66.32%
